# 目录


# 信息熵

定义如下
* 熵 $H(X) = -\sum_x p_x \ln p_x$, 表示收到一个信息所减少系统不确定性的大小。信息熵的来源来自于[香农的信源编码定理](https://en.wikipedia.org/wiki/Shannon%27s_source_coding_theorem#Proof:_Source_coding_theorem)，信息熵是信源无损压缩 bit 数的下界。所以是一条 $X$ 所得到的信息量
* 联合熵 $H(X,Y) = -\sum_{x,y} p_{x,y} \ln p_{x,y}$。表示收到 $(X,Y)$ 所得到的信息量
* 条件熵 $H(Y|X) = H(X,Y) - H(X)=\sum_x p_x H(Y|X=x)$。从后面的表达式可以看出，是在知道 $X$ 后，收到一条 $Y$ 信息所获得的信息量
* 信息增益 $g(D,A)=H(D)-H(D|A)$,其中 $D$ 是结果集合，$A$ 是特征集合。物理解释可以看做在增加 $A$ 后，收到一条信号的信息变化量。变化量越大表示 $A$ 有效的描述了原来的信息源 

# 决策树

算法目标
* ID3, 信息增益 $g(D,A)$
* C4.5, 信息增益率$g_r(D,A) = g(D,A)/H(A)$
* CART, Gini 系数 $Gini(p)=\sum_{k=1}^K p_k(1-p_k) = 1-\sum_{k=1}^Kp_k^2$

纯节点与均节点
* 纯节点。对叶子节点，若 $n_j = n$ 且 $n_1,...,n_{j-1},n_{j+1},...,n_K=0$，其中 $n$ 是叶节点样本数，$n_j$ 为第 $j$ 类样本数，熵为 $H_p0$
* 均节点。$n_1=n_2=...=n_k=n/K$，熵为 $H_u=\ln K$

评价函数为所有叶子节点的熵的样本加权平均
$$
C(T)=\sum_{t\in leaf} N_t \cdot H(t)
$$
越小越好

# 过拟合问题

## 剪枝

## 随机森林

核心目标
* 解决过拟合问题

实验基础
* 人脑的村委会决策机制

核心思想
* 选择n个样本。从原始样本集中取出 n 个样本集合
* 选择k个属性。从原始所有属性中取出 k 个属性集合
* 重复 m 次
* 对这 m 次数据建立决策树，并用投票表决方式得出数据属于哪一类


拉普拉斯平滑

# 样本不平衡问题



In [ ]:
#!/usr/bin/python
# -*- coding:utf-8 -*-

import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree


def iris_type(s):
    it = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
    return it[s]

# 花萼长度、花萼宽度，花瓣长度，花瓣宽度
iris_feature = 'sepal length', 'sepal width', 'petal length', 'petal width'

if __name__ == "__main__":
    path = u'..\\算法讲师\\机器学习升级版\\数据\\4.iris.data'  # 数据文件路径

    # 路径，浮点型数据，逗号分隔，第4列使用函数iris_type单独处理
    data = np.loadtxt(path, dtype=float, delimiter=',', converters={4: iris_type})

    # 将数据的0到3列组成x，第4列得到y
    x, y = np.split(data, (4,), axis=1)
    # 为了可视化，仅使用前两列特征
    x = x[:, :2]

    # 决策树参数估计
    # min_samples_split = 10：如果该结点包含的样本数目大于10，则(有可能)对其分支
    # min_samples_leaf = 10：若将某结点分支后，得到的每个子结点样本数目都大于10，则完成分支；否则，不进行分支
    clf = DecisionTreeClassifier(criterion='entropy', max_depth=30, min_samples_leaf=3)
    dt_clf = clf.fit(x, y)

    # 保存
    # dot -Tpng -o 1.png 1.dot
    f = open("iris_tree.dot", 'w')
    tree.export_graphviz(dt_clf, out_file=f)

    # 画图
    N, M = 500, 500  # 横纵各采样多少个值
    x1_min, x1_max = x[:, 0].min(), x[:, 0].max()  # 第0列的范围
    x2_min, x2_max = x[:, 1].min(), x[:, 1].max()  # 第1列的范围
    t1 = np.linspace(x1_min, x1_max, N)
    t2 = np.linspace(x2_min, x2_max, M)
    x1, x2 = np.meshgrid(t1, t2)  # 生成网格采样点
    x_test = np.stack((x1.flat, x2.flat), axis=1)  # 测试点

    # # 无意义，只是为了凑另外两个维度
    # # 打开该注释前，确保注释掉x = x[:, :2]
    # x3 = np.ones(x1.size) * np.average(x[:, 2])
    # x4 = np.ones(x1.size) * np.average(x[:, 3])
    # x_test = np.stack((x1.flat, x2.flat, x3, x4), axis=1)  # 测试点

    y_hat = dt_clf.predict(x_test)  # 预测值
    y_hat = y_hat.reshape(x1.shape)  # 使之与输入的形状相同
    plt.pcolormesh(x1, x2, y_hat, cmap=plt.cm.Spectral, alpha=0.1)  # 预测值的显示Paired/Spectral/coolwarm/summer/spring/OrRd/Oranges
    plt.scatter(x[:, 0], x[:, 1], c=y, edgecolors='k', cmap=plt.cm.prism)  # 样本的显示
    plt.xlabel(iris_feature[0])
    plt.ylabel(iris_feature[1])
    plt.xlim(x1_min, x1_max)
    plt.ylim(x2_min, x2_max)
    plt.grid()
    plt.show()

    # 训练集上的预测结果
    y_hat = dt_clf.predict(x)
    y = y.reshape(-1)       # 此转置仅仅为了print时能够集中显示
    print y_hat.shape       # 不妨显示下y_hat的形状
    print y.shape
    result = (y_hat == y)   # True则预测正确，False则预测错误
    print y_hat
    print y
    print result
    c = np.count_nonzero(result)    # 统计预测正确的个数
    print c
    print 'Accuracy: %.2f%%' % (100 * float(c) / float(len(result)))


$$
H\left(D\right)=-\sum_{y}p_{y}\ln p_{y}
$$
$$
H\left(D,A\right)=-\sum_{x,y}p_{x,y}\ln p_{x,y}
$$
$$
H\left(D|A\right)=-\sum_{x,y}p_{x,y}\ln p_{x,y}+\sum_{x}p_{x}\ln p_{x}
$$
ID3
\begin{eqnarray*}
g\left(D,A\right) & = & H\left(D\right)-H\left(D|A\right)\\
 & = & -\sum_{y}p_{y}\ln p_{y}-\sum_{x}p_{x}\ln p_{x}+\sum_{x,y}p_{x,y}\ln p_{x,y}\\
 & = & -\sum_{x,y}p_{x,y}\ln p_{y}-\sum_{x,y}p_{x,y}\ln p_{x}+\sum_{x,y}p_{x,y}\ln p_{x,y}\\
 & = & -\sum_{x,y}p_{x,y}\ln\frac{p_{y}p_{x}}{p_{x,y}}\\
 & = & \sum_{x,y}p_{x,y}\ln\frac{p_{y|x}}{p_{y}}
\end{eqnarray*}
条件概率
$$
p_{x|y}=\frac{p_{x,y}}{\sum_{x}p_{x,y}}=\frac{p_{x,y}}{p_{y}}
$$
C4.5
$$
\frac{g\left(D,A\right)}{H\left(D\right)}
$$
